# Messing around with location history

In [1]:
#!pip install HASS-data-detective # Install detective

In [2]:
# !pip install plotly chart-studio swifter

In [3]:
# !pip show HASS-data-detective

In [34]:
from typing import *
import pandas as pd
from sqlalchemy import create_engine, text
from detective.core import stripped_db_url, get_db_type

# Modify hass-data-detective
class HassDatabase:
    """
    Initializing the parser fetches all of the data from the database and
    places it in a master pandas dataframe.
    """

    def __init__(self, url, *, fetch_entities=True):
        """
        Parameters
        ----------
        url : str
            The URL to the database.
        """
        self.url = url
        self.entities = None
        try:
            self.engine = create_engine(url)
            print("Successfully connected to database", stripped_db_url(url))
            if fetch_entities:
                self.fetch_entities()
        except Exception as exc:
            if isinstance(exc, ImportError):
                raise RuntimeError(
                    "The right dependency to connect to your database is "
                    "missing. Please make sure that it is installed."
                )

            print(exc)
            raise

        self.db_type = get_db_type(url)

    def perform_query(self, query, **params):
        """Perform a query."""
        try:
            return self.engine.execute(query, params)
        except:
            print(f"Error with query: {query}")
            raise

    def fetch_entities(self) -> None:
        """Fetch entities for which we have data."""
        query = text(
            """
            SELECT DISTINCT(entity_id) FROM states
            """
        )
        response = self.perform_query(query)

        # Parse the domains from the entities.
        self.entities = [e[0] for e in response]
        print(f"There are {len(self.entities)} entities with data")

    def fetch_all_sensor_data(self, limit=50000, get_attributes=False) -> pd.DataFrame:
        """
        Fetch data for all sensor entities.

        Arguments:
        - limit (default: 50000): Limit the maximum number of state changes loaded.
            If None, there is no limit.
        - get_attributes: If True, LEFT JOIN the attributes table to retrieve event's attributes.
        """
        
        if get_attributes:
            query = """
                SELECT entity_id, state, last_updated
            """
        else:
            query = """
                SELECT entity_id, state, last_updated, shared_attrs
            """
        
        query += "FROM states"
        
        if get_attributes:
            query += """
                LEFT JOIN state_attributes ON states.attributes_id = state_attributes.attributes_id
            """
            
        query += """
            WHERE
                entity_id  LIKE '%sensor%'
            AND
                state NOT IN ('unknown', 'unavailable')
            ORDER BY last_updated DESC
            """
        
        if limit is not None:
            query += f"LIMIT {limit}"
        df = pd.read_sql_query(query, self.url)
        print(f"The returned Pandas dataframe has {df.shape[0]} rows of data.")
        return df

    def fetch_all_data_of(self, sensors: Tuple[str], limit=50000, get_attributes=False) -> pd.DataFrame:
        """
        Fetch data for sensors.

        Arguments:
        - limit (default: 50000): Limit the maximum number of state changes loaded.
            If None, there is no limit.
        - get_attributes: If True, LEFT JOIN the attributes table to retrieve event's attributes.
        """
        sensors_str = str(tuple(sensors))
        if len(sensors) == 1:
            sensors_str = sensors_str.replace(",", "")

        if get_attributes:
            query = """
                SELECT entity_id, state, last_updated, shared_attrs
            """
        else:
            query = """
                SELECT entity_id, state, last_updated
            """

        query += "FROM states"
        
        if get_attributes:
            query += """
                LEFT JOIN state_attributes ON states.attributes_id = state_attributes.attributes_id
            """
            
        query += f"""
            WHERE
                entity_id IN {sensors_str}
            AND
                state NOT IN ('unknown', 'unavailable')
            ORDER BY last_updated DESC
            """

        if limit is not None:
            query += f"LIMIT {limit}"
            
        print(query)
        df = pd.read_sql_query(query, self.url)
        print(f"The returned Pandas dataframe has {df.shape[0]} rows of data.")
        return df


In [35]:
import detective.config as config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import swifter

def my_db_from_hass_config(path=None, **kwargs):
    """Initialize a database from HASS config."""
    if path is None:
        path = config.find_hass_config()

    url = config.db_url_from_hass_config(path)
    return HassDatabase(url, **kwargs)

db = my_db_from_hass_config(fetch_entities=False)

YAML tag !include_dir_merge_list is not supported
YAML tag !include_dir_merge_named is not supported
Successfully connected to database sqlite:////config/home-assistant_v2.db


First we extract all state changes for Gaby and Charles

In [36]:
people = ("person.charles","person.gaby",)

In [39]:
results = db.fetch_all_data_of(people, get_attributes=True)
results


                SELECT entity_id, state, last_updated, shared_attrs
            FROM states
                LEFT JOIN state_attributes ON states.attributes_id = state_attributes.attributes_id
            
            WHERE
                entity_id IN ('person.charles', 'person.gaby')
            AND
                state NOT IN ('unknown', 'unavailable')
            ORDER BY last_updated DESC
            LIMIT 50000
The returned Pandas dataframe has 42847 rows of data.


,entity_id,state,last_updated,shared_attrs
0,person.gaby,home,2022-10-23 10:43:17.976969,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
1,person.gaby,home,2022-10-23 10:38:46.192274,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
2,person.gaby,home,2022-10-23 09:58:18.316593,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
3,person.gaby,home,2022-10-23 09:52:36.118369,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
4,person.gaby,home,2022-10-23 09:52:19.095189,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
...,...,...,...,...
42842,person.gaby,home,2022-05-06 03:38:07.624507,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
42843,person.gaby,home,2022-05-06 03:30:02.878928,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
42844,person.gaby,home,2022-05-06 03:24:58.453512,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
42845,person.gaby,home,2022-05-06 03:20:23.946649,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."


In [7]:
# Copied from HASS-data-detective and modified to join the new attributes table
sensors_str = str(tuple(people))
if len(people) == 1:
    sensors_str = sensors_str.replace(",", "")

query = f"""
    SELECT domain, entity_id, state, last_updated, shared_attrs
    FROM states
    LEFT JOIN state_attributes ON states.attributes_id = state_attributes.attributes_id
    WHERE
        entity_id IN {sensors_str}
    AND
        state NOT IN ('unknown', 'unavailable')
    ORDER BY last_changed DESC
    """

results = pd.read_sql_query(query, db.url)
print(f"{len(results)} results loaded")
results

42845 results loaded


,domain,entity_id,state,last_updated,shared_attrs
0,None,person.charles,home,2022-10-22 04:50:05.728112,"{""editable"":false,""id"":""charles"",""latitude"":51..."
1,None,person.gaby,home,2022-10-22 04:46:59.531755,"{""editable"":false,""id"":""gaby"",""latitude"":51.41..."
2,None,person.gaby,home,2022-10-22 04:47:07.528203,"{""editable"":false,""id"":""gaby"",""latitude"":51.41..."
3,None,person.gaby,home,2022-10-22 04:47:45.386224,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
4,None,person.gaby,home,2022-10-22 04:49:35.753061,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
...,...,...,...,...,...
42840,None,person.gaby,home,2022-05-06 06:43:22.576767,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
42841,None,person.gaby,home,2022-05-06 06:46:22.767125,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
42842,None,person.gaby,home,2022-05-06 06:47:16.641555,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
42843,None,person.gaby,home,2022-05-06 06:47:34.924815,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."


In [8]:
results.last_changed = pd.to_datetime(results["last_updated"])
results.set_index('last_updated', inplace=True)
results.index.rename("time", inplace=True)
results.sort_index(inplace=True)
results

/tmp/ipykernel_981/2555479191.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  results.last_changed = pd.to_datetime(results["last_updated"])


,domain,entity_id,state,shared_attrs
time,,,,
2022-05-06 03:15:07.110833,None,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
2022-05-06 03:20:23.946649,None,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
2022-05-06 03:24:58.453512,None,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
2022-05-06 03:30:02.878928,None,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
2022-05-06 03:38:07.624507,None,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
...,...,...,...,...
2022-10-23 09:10:23.875677,None,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
2022-10-23 09:46:38.078987,None,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."
2022-10-23 09:52:19.095189,None,person.gaby,home,"{""editable"":false,""id"":""gaby"",""latitude"":51.40..."


In [9]:
import yaml
l = lambda s : yaml.safe_load(s)
results["parsed_attrs"] = results.loc[:, "shared_attrs"].swifter.apply(l)

Pandas Apply:   0%|          | 0/42845 [00:00<?, ?it/s]


KeyboardInterrupt



In [ ]:
results['latitude'] = results['parsed_attrs'].apply(lambda i : i['latitude'])
results['longitude'] = results['parsed_attrs'].apply(lambda i : i['longitude'])

In [ ]:
import matplotlib.pyplot as plt

for person in people:
    plt.figure()
    k = (results["entity_id"] == person) & (results["longitude"] > -20)
    plt.scatter(results.loc[k, 'longitude'], results.loc[k, 'latitude'])
    plt.legend([person])
# results.plot.scatter(x="longitude", y="latitude", alpha=0.4, label="entity_id");

In [ ]:
out = results.loc[:,['entity_id', 'latitude', 'longitude']]
out

In [ ]:
out.to_csv("locations.csv")

In [ ]:
pd.read_csv("locations.csv")

In [ ]:
# import plotly
# import chart_studio.plotly as py
# import plotly.graph_objs as go
# plotly.offline.init_notebook_mode()

# coords = pd.concat([results['latitude'], results['longitude']], axis=1)
# coords = coords.sample(frac=0.1, replace=True)

# colors = ['rgb(239,243,255)','rgb(189,215,231)','rgb(107,174,214)','rgb(33,113,181)']

# points = go.Scattergeo(
#     lon = coords['longitude'],
#     lat = coords['latitude']
# )


# layout = go.Layout(
#     title = 'Hey look! It\'s a scatter plot on a map!',
#     geo = dict(
#         # resolution = 110,
#         scope = 'europe',
#         # showframe = True,
#         # showcoastlines = True,
#         # showland = True,
#         # landcolor = "rgb(229, 229, 229)",
#         # countrycolor = "rgb(255, 255, 255)" ,
#         # coastlinecolor = "rgb(255, 255, 255)",
#         # projection = dict(
#         #     type = 'mercator'
#         # ),
#         # lonaxis = dict( range= [ -124.0, -113.0 ] ),
#         # lataxis = dict( range= [ 32.0, 43.0 ] ),
        
#     ),
#     legend = dict(
#            traceorder = 'reversed'
#     )
# )


# fig = go.Figure(layout=layout, data=points)
# plotly.offline.iplot(fig, validate=False)

In [ ]:
# import plotly.express as px
# df = px.data.gapminder().query("year == 2007")
# fig = px.scatter_geo(df, locations="iso_alpha",
#                      size="pop", # size of markers, "pop" is one of the columns of gapminder
#                      )
# fig.show()

In [ ]:
# coords

In [ ]:
# k = results["longitude"] > -20
# filtered_results = results.loc[k, :]

# import plotly.express as px


# fig = px.scatter_geo(filtered_results, lat="latitude", lon="longitude", scope="europe")
# fig.show()